In [1]:
#%cd /content/drive/My Drive/SegFormer
import argparse
import logging
import os
import random
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from utils import clip_gradient, AvgMeter

from glob import glob
from skimage.io import imread
import matplotlib.pyplot as plt
import pandas as pd
from collections import OrderedDict
from torch.autograd import Variable
from datetime import datetime
import torch.nn.functional as F
import cv2
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
import math
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse
import copy
import os
import os.path as osp
import time

import mmcv
import torch
from mmcv.runner import init_dist
from mmcv.utils import Config, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

In [3]:
from mmseg.models.segmentors import CaraSegUPer_ver2 as UNet
from mmseg.models.segmentors import CaraSegUPer_wBiFPN as Net
from mmseg.models.segmentors import CaraSegUPer_wBiRAFPN as Net2

In [4]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, img_paths, mask_paths, aug=True, transform=None):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.aug = aug
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        mask_path = self.mask_paths[idx]
        # image = imread(img_path)
        # mask = imread(mask_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, 0)
        # name = self.img_paths[idx].split('/')[-1]

        if self.transform is not None:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 
        else:
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 

        image = image.astype('float32') / 255
        image = image.transpose((2, 0, 1))

        mask = mask[:,:,np.newaxis]
        mask = mask.astype('float32') / 255
        mask = mask.transpose((2, 0, 1))

        return np.asarray(image), np.asarray(mask)

In [5]:
from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  possible_positives = torch.sum(torch.round(torch.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  predicted_positives = torch.sum(torch.round(torch.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def dice_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return recall*precision/(recall+precision-recall*precision +K.epsilon())

In [6]:
class FocalLossV1(nn.Module):

    def __init__(self,
                 alpha=0.25,
                 gamma=2,
                 reduction='mean',):
        super(FocalLossV1, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.crit = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, label):
        # compute loss
        logits = logits.float() # use fp32 if logits is fp16
        with torch.no_grad():
            alpha = torch.empty_like(logits).fill_(1 - self.alpha)
            alpha[label == 1] = self.alpha

        probs = torch.sigmoid(logits)
        pt = torch.where(label == 1, probs, 1 - probs)
        ce_loss = self.crit(logits, label.float())
        loss = (alpha * torch.pow(1 - pt, self.gamma) * ce_loss)
        if self.reduction == 'mean':
            loss = loss.mean()
        if self.reduction == 'sum':
            loss = loss.sum()
        return loss
class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, eps=1):
        super().__init__()

        self.eps = eps
        self.alpha = alpha

    def forward(self, pred, label):
        """
        Forward function
        :param pred: Prediction tensor containing raw network outputs (no logit) (B x C x H x W)
        :param label: Label mask tensor (B x C x H x W)
        """
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])
        return 1 - torch.mean(
            (true_pos + self.eps)
            / (
                true_pos
                + self.alpha * false_neg
                + (1 - self.alpha) * false_pos
                + self.eps
            )
        )

class FocalTverskyLoss(TverskyLoss):
    def __init__(self, gamma=4 / 3, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma

    def forward(self, pred, label):
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])

        t = (true_pos + self.eps) / (
            true_pos + self.alpha * false_neg + (1 - self.alpha) * false_pos + self.eps
        )

        x = torch.pow(1 - t, 1 / self.gamma)

        return x #torch.sum(x)
def structure_loss_v2(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))

    
    wiou = FocalTverskyLoss()(pred,mask)
    return (wfocal + wiou).mean()
def structure_loss(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask)*weit).sum(dim=(2, 3))
    union = ((pred + mask)*weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1)/(union - inter+1)
    return (wfocal + wiou).mean()

In [7]:
def train(train_loader, model, optimizer, epoch, lr_scheduler, deep=False):
    model.train()
    # ---- multi-scale training ----
    size_rates = [256/384, 1, 512/384]
    loss_record = AvgMeter()
    dice, iou = AvgMeter(), AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        if epoch <= 1:
                optimizer.param_groups[0]["lr"] = (epoch * i) / (1.0 * total_step) * init_lr
        else:
            lr_scheduler.step()

        for rate in size_rates: 
            optimizer.zero_grad()
            # ---- data prepare ----
            images, gts = pack
            images = Variable(images).cuda()
            gts = Variable(gts).cuda()
            # ---- rescale ----
            trainsize = int(round(trainsize_init*rate/32)*32)
            images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            # ---- forward ----
            map4, map3, map2, map1 = model(images)
           # print(map4.shape, map3.shape, map2.shape, map1.shape)
            map1 = F.upsample(map1, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map2 = F.upsample(map2, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map3 = F.upsample(map3, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map4 = F.upsample(map4, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            loss = structure_loss(map1, gts) + structure_loss(map2, gts) + structure_loss(map3, gts) + structure_loss(map4, gts)
            # ---- metrics ----
            dice_score = dice_m(map2, gts)
            iou_score = iou_m(map2, gts)
            # ---- backward ----
            loss.backward()
            clip_gradient(optimizer, clip)
            optimizer.step()
            # ---- recording loss ----
            if rate == 1:
                loss_record.update(loss.data, batchsize)
                dice.update(dice_score.data, batchsize)
                iou.update(iou_score.data, batchsize)

        # ---- train visualization ----
        if i == total_step:
            print('{} Training Epoch [{:03d}/{:03d}], '
                  '[loss: {:0.4f}, dice: {:0.4f}, iou: {:0.4f}]'.
                  format(datetime.now(), epoch, num_epochs,\
                         loss_record.show(), dice.show(), iou.show()))

    ckpt_path = save_path + 'last.pth'
    print('[Saving Checkpoint:]', ckpt_path)
    checkpoint = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': lr_scheduler.state_dict()
    }
    torch.save(checkpoint, ckpt_path)

    log = OrderedDict([
        ('loss', loss_record.show()), ('dice', dice.show()), ('iou', iou.show()),
    ])

    return log

In [8]:
def recall_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def dice_np(y_true, y_pred):
    precision = precision_np(y_true, y_pred)
    recall = recall_np(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_np(y_true, y_pred):
    intersection = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    union = np.sum(y_true)+np.sum(y_pred)-intersection
    return intersection/(union+K.epsilon())

def get_scores(gts, prs):
    mean_precision = 0
    mean_recall = 0
    mean_iou = 0
    mean_dice = 0
    for gt, pr in zip(gts, prs):
        mean_precision += precision_np(gt, pr)
        mean_recall += recall_np(gt, pr)
        mean_iou += iou_np(gt, pr)
        mean_dice += dice_np(gt, pr)

    mean_precision /= len(gts)
    mean_recall /= len(gts)
    mean_iou /= len(gts)
    mean_dice /= len(gts)        
    
    print(f"scores: dice={mean_dice}, miou={mean_iou}, precision={mean_precision}, recall={mean_recall}")

    return (mean_iou, mean_dice, mean_precision, mean_recall)



def inference(model,writer=None,epoch=None):
    print("#"*20)
    model.eval()
    dataset_names = ['Kvasir', 'CVC-ClinicDB', 'CVC-ColonDB', 'CVC-300', 'ETIS-LaribPolypDB']
    for dataset_name in dataset_names:
        data_path = f'../dataset/scenario_4/all_datasets//TestDataset/{dataset_name}'
        print(data_path)
        X_test = glob('{}/images/*'.format(data_path))
        X_test.sort()
        y_test = glob('{}/masks/*'.format(data_path))
        y_test.sort()

        test_dataset = Dataset(X_test, y_test)
        test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=1,
            shuffle=False,
            pin_memory=True,
            drop_last=False)

        print('Dataset_name:', dataset_name)
        tp_all = 0
        fp_all = 0
        fn_all = 0
        mean_iou = 0
        gts = []
        prs = []
        losses = []
        for i, pack in enumerate(test_loader, start=1):
            image, gt_ = pack
            # name = name[0]
            gt = gt_[0][0]
            gt = np.asarray(gt, np.float32)
            
            image = image.cuda()
            #print(image.shape)
            res, res2, res3, res4 = model(image)
            res = F.upsample(res, size=gt.shape, mode='bilinear', align_corners=False)
#             loss = structure_loss_v2(res.cpu(), gt_)
#             losses.append(loss.cpu().detach().numpy().squeeze())
            #res = res.sigmoid().data.cpu().numpy().squeeze()
            #print(res.shape)
#             res = (res - res.min()) / (res.max() - res.min() + 1e-8)
#             pr = res.round()
            
            image_ = torch.flip(image,[3])
            res_, res2, res3, res4 = model(image_)
            res_ = F.upsample(torch.flip(res_,[3]), size=gt.shape, mode='bilinear', align_corners=False)
#             loss = structure_loss_v2(res.cpu(), gt_)
#             losses.append(loss.cpu().detach().numpy().squeeze())

            res_ = (res_+res)/2.
            res_ = res_.sigmoid().data.cpu().numpy().squeeze()
            
            res_ = (res_ - res_.min()) / (res_.max() - res_.min() + 1e-8)
            pr = res_.round()
            
           # pr = (pr+pr_)/2
            # cv2.imwrite(os.path.join(save_path, dataset_name, name), res)
            gts.append(gt)
            prs.append(pr)
        mean_iou, mean_dice, mean_precision, mean_recall = get_scores(gts, prs)
#         writer.add_scalar(dataset_name + ' val loss',
#                             np.mean(losses),
#                             epoch )
#         writer.add_scalar(dataset_name + ' val dice',
#                             mean_dice,
#                             epoch )
#         writer.add_scalar(dataset_name + ' val iou',
#                             mean_iou,
#                             epoch )
    print("#"*20)

In [9]:
#from torchvision.transforms.functional import *
#import torchvision.transforms as transforms
from albumentations.augmentations.geometric import  resize,rotate
import albumentations.augmentations.crops.transforms as crop
import albumentations.augmentations.transforms as transforms
train_transform = Compose([
            rotate.RandomRotate90(),
            transforms.Flip(),
            transforms.HueSaturationValue(),
            transforms.RandomBrightnessContrast(),
            transforms.GaussianBlur(),
            transforms.Transpose(),
            OneOf([
                crop.RandomCrop(224, 224, p=1),
                crop.CenterCrop(224, 224, p=1)
            ], p=0.2),
            resize.Resize(384, 384)
        ], p=0.5)

In [10]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
#writer = SummaryWriter('logsRA_new/lan1')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_5_new_bottleneck_5_block'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)#train_transform
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                   compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()
model.load_state_dict(torch.load(save_path+"last.pth")['state_dict'])
inference(model,None,epoch=0)


Save path existed


2022-09-03 20:51:22,175 - mmseg - INFO - Use load_from_local loader
2022-09-03 20:51:22,336 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir


/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/torch/nn/functional.py:3509: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


scores: dice=0.9177597212096649, miou=0.8706346858337254, precision=0.9511124297846772, recall=0.9023427061384109
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9398172788396015, miou=0.8936910266096771, precision=0.9389807611021882, recall=0.9503455563889138
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8288802747386259, miou=0.7556776595843535, precision=0.842887186594385, recall=0.8551862286673744
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9004449136721387, miou=0.8368018682404467, precision=0.869347848226091, recall=0.9590263218734916
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.8216654481558809, miou=0.7430856593944961, precision=0.7714600901569987, recall=0.9336874259018051
####################


In [13]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new/lan2')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_2_new'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    #if epoch >= num_epochs-20:
    inference(model,writer,epoch)

2022-05-29 15:16:50,565 - mmseg - INFO - Use load_from_local loader
2022-05-29 15:16:50,628 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
scores: dice=0.5752564583190956, miou=0.47297519919994174, precision=0.5318790001217258, recall=0.7245332819707915
####################
2022-05-29 15:25:54.098304 Training Epoch [002/030], [loss: 1.5286, dice: 0.8434, iou: 0.7357]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.8679320809857195, miou=0.7990174258914525, precision=0.930059565492362, recall=0.8537212615107285
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.8332893057064029, miou=0.7549491751669143, precision=0.8596909652154653, recall=0.8598516399232371
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.757731820688683, miou=0.662040284995784, precision=0.7644910267318678, recall=0.82134780097615
../dataset/scenario_4/all_datasets//TestDataset/

scores: dice=0.749248647131588, miou=0.6565763672946182, precision=0.7089614866406695, recall=0.8325424145378296
####################
2022-05-29 16:01:26.186038 Training Epoch [009/030], [loss: 0.8730, dice: 0.9270, iou: 0.8667]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.899051474487368, miou=0.8421613083545555, precision=0.9561248423215147, recall=0.8748076100327437
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9019318154405325, miou=0.8485380108231272, precision=0.9193848966314491, recall=0.903989367670316
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7590509738694918, miou=0.6799772047280096, precision=0.7907563587308273, recall=0.7838493174956613
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.89429356423

scores: dice=0.8067733972645755, miou=0.7364785579344159, precision=0.7837762574881064, recall=0.8951408864216204
####################
2022-05-29 16:36:56.748157 Training Epoch [016/030], [loss: 0.7405, dice: 0.9316, iou: 0.8791]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9082486982020325, miou=0.8539823597425662, precision=0.9425951746047059, recall=0.9030223522690163
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.8985546173125148, miou=0.8466421831930416, precision=0.8905581839942324, recall=0.9470024014397792
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8157076085203488, miou=0.7401110534562056, precision=0.7974308065518063, recall=0.8886638646249666
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.88363977

scores: dice=0.823537409812151, miou=0.7427094623009338, precision=0.7780846817082425, recall=0.9304027739954289
####################
2022-05-29 17:12:27.700745 Training Epoch [023/030], [loss: 0.6675, dice: 0.9419, iou: 0.8946]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9307890436631421, miou=0.8830956431241161, precision=0.9537556817705639, recall=0.9236523825952975
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9244732365581361, miou=0.8766354064475136, precision=0.9315276096881121, recall=0.939133307711418
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8086541677071535, miou=0.7362257356440098, precision=0.820965027256035, recall=0.841561066594146
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.899889110141

scores: dice=0.8260402339357011, miou=0.7474664954387881, precision=0.7839604481546416, recall=0.9345858909402448
####################
2022-05-29 17:47:58.708128 Training Epoch [030/030], [loss: 0.6164, dice: 0.9457, iou: 0.9022]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9229199554624031, miou=0.87424620327455, precision=0.9428957214325123, recall=0.9245320419560877
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9280427396386309, miou=0.8823540806059694, precision=0.9334477450383293, recall=0.9442061809973371
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8153160308400447, miou=0.7393184222934512, precision=0.8166945372970629, recall=0.8695519348212724
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8910303949

In [14]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new/lan3')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_3_new'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    #if epoch >= num_epochs-20:
    inference(model,writer,epoch)

2022-05-29 17:49:05,615 - mmseg - INFO - Use load_from_local loader
2022-05-29 17:49:05,678 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-05-29 17:53:04.533293 Training Epoch [001/030], [loss: 2.3603, dice: 0.7873, iou: 0.6546]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.8297808516162394, miou=0.7423767891933271, precision=0.8586385149514064, recall=0.8478605235122495
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.7788302891193236, miou=0.6819303650190695, precision=0.7648956961164444, recall=0.8605755248691115
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.6903173024309088, miou=0.5905943055593145, precision=0.6784225608853147, recall=0.8018613434165478
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8217307683750157, miou=0.7321511444979365, precision=0.7507765814353333, recall=0.9729

scores: dice=0.7842273163018352, miou=0.7075562741909052, precision=0.7407419222752395, recall=0.9007185088035031
####################
2022-05-29 18:28:38.145404 Training Epoch [008/030], [loss: 0.9787, dice: 0.9031, iou: 0.8275]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.8765833873994748, miou=0.811579394593863, precision=0.9360543070727446, recall=0.8534847800504995
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.8827021081543022, miou=0.8220888998417091, precision=0.8756857137811613, recall=0.9191825021276477
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7940802185910647, miou=0.7087442959709678, precision=0.7809892366653176, recall=0.872747290724333
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8822784158

scores: dice=0.7785153054245243, miou=0.6964843946414465, precision=0.7144894256636956, recall=0.9264040450244782
####################
2022-05-29 19:04:13.972062 Training Epoch [015/030], [loss: 0.7485, dice: 0.9453, iou: 0.8979]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9122196256763644, miou=0.861807846408994, precision=0.9488903164385495, recall=0.8964367072638142
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9031357837095759, miou=0.8535554950292902, precision=0.9142490680151042, recall=0.9314378328046307
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.801531231553794, miou=0.7328701766474407, precision=0.8142227714503741, recall=0.8501674861412225
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9079947496

scores: dice=0.8155199657416013, miou=0.7362103049789899, precision=0.760666977387234, recall=0.9368987988660303
####################
2022-05-29 19:39:48.272457 Training Epoch [022/030], [loss: 0.6335, dice: 0.9359, iou: 0.8874]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9111919049968271, miou=0.8599023657985514, precision=0.9420987515424766, recall=0.9024876029328989
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9150110409491747, miou=0.8682080508486871, precision=0.9007165605454795, recall=0.9580053842690129
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.827378776973896, miou=0.7532648519487558, precision=0.8237151440807959, recall=0.8850031322461713
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8824553088

scores: dice=0.8301076441520335, miou=0.7564224513596101, precision=0.7878105479082163, recall=0.9294892543497252
####################
2022-05-29 20:15:22.886666 Training Epoch [029/030], [loss: 0.5975, dice: 0.9613, iou: 0.9259]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9160974779367269, miou=0.8665148646140863, precision=0.9468858816400453, recall=0.9065707317105478
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9343629507239283, miou=0.890625692055573, precision=0.9276714083693689, recall=0.9555730545822188
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.832096198481804, miou=0.7584224779509123, precision=0.838790158519808, recall=0.8724519741838733
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.90500172230

In [15]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new/lan4')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_4_new'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    #if epoch >= num_epochs-20:
    inference(model,writer,epoch)

2022-05-29 20:21:34,792 - mmseg - INFO - Use load_from_local loader
2022-05-29 20:21:34,855 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-05-29 20:25:33.532775 Training Epoch [001/030], [loss: 2.4423, dice: 0.7646, iou: 0.6276]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.780812689817947, miou=0.6805849091766746, precision=0.7392437151149744, recall=0.9167711124269609
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.6885482224031192, miou=0.5702823432409472, precision=0.612912758442019, recall=0.9073353469169917
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.6455491643869201, miou=0.5368633194511006, precision=0.5814197176940428, recall=0.8815075170082783
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.7454458707385997, miou=0.6316887236742215, precision=0.6366293621692348, recall=0.988186

scores: dice=0.7139008234954005, miou=0.6132472440203833, precision=0.6270310057416764, recall=0.9155947599245665
####################
2022-05-29 21:01:04.995541 Training Epoch [008/030], [loss: 1.0641, dice: 0.8856, iou: 0.8049]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.8779688522730656, miou=0.8162813624769463, precision=0.9401857243054745, recall=0.8554311690458266
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.8630686554695894, miou=0.8019379104093327, precision=0.8966577090575454, recall=0.8915680441586116
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7711995015927435, miou=0.6951349391452676, precision=0.7697258989681826, recall=0.8475330441238588
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.86884010

scores: dice=0.7610596735212777, miou=0.6883227404079406, precision=0.7262211525322186, recall=0.864723701941074
####################
2022-05-29 21:36:37.061365 Training Epoch [015/030], [loss: 0.8408, dice: 0.9122, iou: 0.8508]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9158360718982437, miou=0.8646034294811752, precision=0.9434035529875878, recall=0.9143050880070536
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9271617315091784, miou=0.8781249519727163, precision=0.9225244636678802, recall=0.9500473852851704
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7929488545291596, miou=0.7174165579191751, precision=0.7963427960638622, recall=0.8643026163967983
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.889970890

scores: dice=0.8130797029602432, miou=0.7338830236151285, precision=0.7613220087052412, recall=0.9223045254211379
####################
2022-05-29 22:17:13.549648 Training Epoch [023/030], [loss: 0.5827, dice: 0.9606, iou: 0.9249]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9263601909727813, miou=0.8773845337516119, precision=0.941644519934475, recall=0.9293416146643245
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9207997553310662, miou=0.8713796286710638, precision=0.9097521503876544, recall=0.9561976305651277
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8168337858630426, miou=0.7354615539403764, precision=0.8209584310837583, recall=0.8617611094375226
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.888646696

scores: dice=0.8413551540151559, miou=0.7667080540797022, precision=0.801767749861486, recall=0.9237044334940179
####################
2022-05-29 22:52:45.831106 Training Epoch [030/030], [loss: 0.5584, dice: 0.9641, iou: 0.9316]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9241830079941751, miou=0.8754654413761584, precision=0.9497905799538351, recall=0.9180809415511085
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9397731478259026, miou=0.8944386375587079, precision=0.9357029918753103, recall=0.9537464130780605
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.829304218004663, miou=0.7500131472442044, precision=0.8513553839682957, recall=0.8590107451157746
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9070519303

In [16]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new/lan5')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_5_new'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    #if epoch >= num_epochs-20:
    inference(model,writer,epoch)

2022-05-29 22:53:52,649 - mmseg - INFO - Use load_from_local loader
2022-05-29 22:53:52,712 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-05-29 22:57:50.962665 Training Epoch [001/030], [loss: 2.4138, dice: 0.7392, iou: 0.5974]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.8127177458643692, miou=0.7179376410912846, precision=0.8003635887248212, recall=0.8797287192241098
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.7576131420704683, miou=0.6497947688820543, precision=0.7310881420959879, recall=0.8563705312031191
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.690098576237403, miou=0.5782466847222227, precision=0.6761737681846111, recall=0.7889606347057934
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.7785089540014472, miou=0.6543979192785742, precision=0.6640354323378042, recall=0.98701

scores: dice=0.753374214584186, miou=0.6792254620545827, precision=0.7068310376442131, recall=0.8622846160447273
####################
2022-05-29 23:33:23.054264 Training Epoch [008/030], [loss: 0.9633, dice: 0.8991, iou: 0.8239]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.8856806728128866, miou=0.8289005170904206, precision=0.9372635286512617, recall=0.8684946663122771
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.8961607810355228, miou=0.8406599361113148, precision=0.9090294500083128, recall=0.9057685740367745
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7788219316925701, miou=0.7024467650528936, precision=0.7893147853722156, recall=0.8283328536431321
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.874651205

2022-05-30 00:08:54.488798 Training Epoch [015/030], [loss: 0.6905, dice: 0.9519, iou: 0.9090]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9016431079347136, miou=0.8464331928544688, precision=0.9486298029011955, recall=0.8899778900297619
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9238789185303239, miou=0.8747389308454391, precision=0.928401050070977, recall=0.94064161427068
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8108259222639214, miou=0.7324577466000196, precision=0.837037869617859, recall=0.8298989297068693
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9099090065779396, miou=0.8460815041662685, precision=0.8818446386496817, recall=0.9560477100411597
../dataset/scenario_4/all_datasets//TestDataset/

2022-05-30 00:44:26.457062 Training Epoch [022/030], [loss: 0.6282, dice: 0.9594, iou: 0.9230]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9127138520721254, miou=0.8592077942810014, precision=0.9341750044764627, recall=0.9121115507311134
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.917396277608155, miou=0.8663156003476764, precision=0.9144806552540375, recall=0.9444143435374383
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8333578092638142, miou=0.7563080083718319, precision=0.8491979428722254, recall=0.8530130960898046
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9037492238354452, miou=0.8399861110442799, precision=0.8744611835753994, recall=0.9572420191049195
../dataset/scenario_4/all_datasets//TestDatas

2022-05-30 01:19:58.049756 Training Epoch [029/030], [loss: 0.5613, dice: 0.9589, iou: 0.9224]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9201630570072392, miou=0.868788290832007, precision=0.9414308746991762, recall=0.9207707889404556
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9436096505891519, miou=0.8975595940335359, precision=0.937303509122143, recall=0.954586807859931
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8249964581047613, miou=0.7479326215581429, precision=0.8316413783729343, recall=0.8637014646549246
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9000732159371242, miou=0.8343136253188038, precision=0.8654194959241635, recall=0.9605842179323629
../dataset/scenario_4/all_datasets//TestDataset

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() )#if p.requires_grad)
from pthflops import count_ops
print(count_parameters(model))
x = torch.rand(1,3,384,384)
#count_ops(model, x,mode='jit')
from fvcore.nn import FlopCountAnalysis
flops = FlopCountAnalysis(model, x)
flops.total()

97729974


/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:98: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:104: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  kv

53346719124

In [12]:
from torchsummary import summary


summary(model, (3, 384, 384))

Layer (type:depth-idx)                                  Output Shape              Param #
├─mit_b3: 1-1                                           [-1, 64, 96, 96]          --
|    └─OverlapPatchEmbed: 2-1                           [-1, 9216, 64]            --
|    |    └─Conv2d: 3-1                                 [-1, 64, 96, 96]          9,472
|    |    └─LayerNorm: 3-2                              [-1, 9216, 64]            128
|    └─ModuleList: 2                                    []                        --
|    |    └─Block: 3-3                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-4                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-5                                  [-1, 9216, 64]            314,880
|    └─LayerNorm: 2-2                                   [-1, 9216, 64]            128
|    └─OverlapPatchEmbed: 2-3                           [-1, 2304, 128]           --
|    |    └─Conv2d: 3-6                 

/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Layer (type:depth-idx)                                  Output Shape              Param #
├─mit_b3: 1-1                                           [-1, 64, 96, 96]          --
|    └─OverlapPatchEmbed: 2-1                           [-1, 9216, 64]            --
|    |    └─Conv2d: 3-1                                 [-1, 64, 96, 96]          9,472
|    |    └─LayerNorm: 3-2                              [-1, 9216, 64]            128
|    └─ModuleList: 2                                    []                        --
|    |    └─Block: 3-3                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-4                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-5                                  [-1, 9216, 64]            314,880
|    └─LayerNorm: 2-2                                   [-1, 9216, 64]            128
|    └─OverlapPatchEmbed: 2-3                           [-1, 2304, 128]           --
|    |    └─Conv2d: 3-6                 